Semana 9 - Reto Calidad del agua

Lázaro Lara Martínez. Matricula A01793198
José Mtanous Treviño. Matricula A00169781

Ciencia y Analítica de datos.

Profesor Titular.
Profesor Tutor. Mtro. Mario Alberto Solano Saldaña

14/Noviembre/2022


In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import zscore
import geopandas
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
# activate plot theme
import qeds
qeds.themes.mpl_style();
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.compose import TransformedTargetRegressor

from sklearn.metrics import f1_score, precision_score,confusion_matrix, plot_roc_curve,RocCurveDisplay,recall_score, make_scorer,accuracy_score



In [15]:
# cargamos nuestro dataset y lo inspeccionamos rapídamente
path_calidad_subterranea = '../Reto/datos/Escalas_subterranea_clean.csv'
dfCalidadSubterranea = pd.read_csv(path_calidad_subterranea)

In [16]:
dfCalidadSubterranea.head()

,Unnamed: 0,SITIO,ORGANISMO_DE_CUENCA,ESTADO,MUNICIPIO,ACUIFERO,SUBTIPO,LONGITUD,LATITUD,ALC_mg/L,...,CUMPLE_CON_DUR,CUMPLE_CON_CF,CUMPLE_CON_NO3,CUMPLE_CON_AS,CUMPLE_CON_CD,CUMPLE_CON_CR,CUMPLE_CON_HG,CUMPLE_CON_PB,CUMPLE_CON_MN,CUMPLE_CON_FE
0,0,POZO SAN GIL,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,ASIENTOS,VALLE DE CHICALOTE,POZO,-102.02210,22.20887,229.990,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
1,1,POZO R013 CAÑADA HONDA,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,AGUASCALIENTES,VALLE DE CHICALOTE,POZO,-102.20075,21.99958,231.990,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
2,2,POZO COSIO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,COSIO,VALLE DE AGUASCALIENTES,POZO,-102.28801,22.36685,204.920,...,SI,SI,SI,NO,SI,SI,SI,SI,SI,SI
3,3,POZO EL SALITRILLO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,RINCON DE ROMOS,VALLE DE AGUASCALIENTES,POZO,-102.29449,22.18435,327.000,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
4,4,RANCHO EL TECOLOTE,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LA PAZ,TODOS SANTOS,POZO,-110.24480,23.45138,309.885,...,SI,SI,NO,SI,SI,SI,SI,SI,SI,SI


Selecciona tus variables independientes X y dependiente Y (semáforo)

In [17]:
#Asignación de variables de entrada y salida
X=dfCalidadSubterranea.loc[:,['SITIO', 'ORGANISMO_DE_CUENCA', 'ESTADO', 'MUNICIPIO','ACUIFERO', 'SUBTIPO', 'LONGITUD', 'LATITUD', 'ALC_mg/L', 'CALIDAD_ALC',
       'CONDUCT_mS/cm', 'CALIDAD_CONDUC', 'SDT_M_mg/L', 'CALIDAD_SDT_ra','CALIDAD_SDT_salin', 'FLUORUROS_mg/L', 'CALIDAD_FLUO', 'DUR_mg/L',
       'CALIDAD_DUR', 'COLI_FEC_NMP/100_mL', 'CALIDAD_COLI_FEC', 'N_NO3_mg/L','CALIDAD_N_NO3', 'AS_TOT_mg/L', 'CALIDAD_AS', 'CD_TOT_mg/L',
       'CALIDAD_CD', 'CR_TOT_mg/L', 'CALIDAD_CR', 'HG_TOT_mg/L', 'CALIDAD_HG','PB_TOT_mg/L', 'CALIDAD_PB', 'MN_TOT_mg/L', 'CALIDAD_MN', 'FE_TOT_mg/L',
       'CALIDAD_FE', 'CONTAMINANTES', 'CUMPLE_CON_ALC','CUMPLE_CON_COND', 'CUMPLE_CON_SDT_ra', 'CUMPLE_CON_SDT_salin',
       'CUMPLE_CON_FLUO', 'CUMPLE_CON_DUR', 'CUMPLE_CON_CF', 'CUMPLE_CON_NO3','CUMPLE_CON_AS', 'CUMPLE_CON_CD', 'CUMPLE_CON_CR', 'CUMPLE_CON_HG',
       'CUMPLE_CON_PB', 'CUMPLE_CON_MN', 'CUMPLE_CON_FE']]
Y= dfCalidadSubterranea.loc[:,['SEMAFORO']]
dfCalidadSubterranea.shape

(1068, 55)

Cambia a label encoding el semáforo, ej, de ["clase 1", "clase 2", "clase 3"] a [ 1,2,3]

In [50]:
ohe = OneHotEncoder()

ct = ColumnTransformer([('one-hot-encoder', OneHotEncoder(), ['SEMAFORO'])], remainder='passthrough')
y = pd.DataFrame(ct.fit_transform(dfCalidadSubterranea)).loc[:,[0,1,2]]
y
###Que hacemos con el ?

,0,1,2
0,0.0,0.0,1.0
1,0.0,0.0,1.0
2,0.0,1.0,0.0
3,0.0,0.0,1.0
4,0.0,1.0,0.0
...,...,...,...
1063,0.0,1.0,0.0
1064,0.0,1.0,0.0
1065,0.0,1.0,0.0
1066,0.0,0.0,1.0


In [51]:
#Particiòn de datos de entrenamiento y validación. 100 datos de prueba representa el 15%
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y, train_size=0.80, random_state=0)

In [52]:
#Verificación del tamaño de conjuntos de entrenamiento y prueba
print(Xtest.shape)
print(Xtrain.shape)

(214, 53)
(854, 53)


In [53]:
X.describe()

,LONGITUD,LATITUD,ALC_mg/L,CONDUCT_mS/cm,SDT_M_mg/L,FLUORUROS_mg/L,DUR_mg/L,COLI_FEC_NMP/100_mL,N_NO3_mg/L,AS_TOT_mg/L,CD_TOT_mg/L,CR_TOT_mg/L,HG_TOT_mg/L,PB_TOT_mg/L,MN_TOT_mg/L
count,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000
mean,-101.891007,23.163618,235.633759,1138.953013,896.101567,1.075600,347.938073,355.490356,4.319759,0.019618,0.003030,0.013276,0.000557,0.005282,0.072478
std,6.703263,3.887670,116.655014,1242.056683,2748.950627,1.924278,359.500870,2052.457014,8.341222,0.035209,0.000894,0.154391,0.000467,0.003254,0.376512
min,-116.664250,14.561150,26.640000,50.400000,25.000000,0.200000,20.000000,1.100000,0.020000,0.010000,0.003000,0.005000,0.000500,0.005000,0.001500
25%,-105.388865,20.212055,164.048750,505.500000,337.700000,0.267175,121.274100,1.100000,0.650932,0.010000,0.003000,0.005000,0.000500,0.005000,0.001500
50%,-102.174180,22.617190,215.825000,822.500000,551.400000,0.503500,245.662350,1.100000,2.082916,0.010000,0.003000,0.005000,0.000500,0.005000,0.001500
75%,-98.974716,25.510285,292.423750,1321.250000,915.900000,1.139850,453.930000,13.250000,5.200047,0.010000,0.003000,0.005000,0.000500,0.005000,0.009947
max,-86.864120,32.677713,1650.000000,18577.000000,82170.000000,34.803300,3810.692200,24196.000000,121.007813,0.452200,0.032110,5.003200,0.014150,0.080900,8.982000


In [54]:
y.describe()

,SEMAFORO
count,1068
unique,3
top,Verde
freq,434


In [40]:
one_hot_transformer= Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])
transformer = ColumnTransformer(
    transformers=[
       ('Semaforo', one_hot_transformer, 'SEMAFORO'),
    ],
    remainder = 'passthrough'
)

In [45]:
#Modelo Dummy
modelo_Dummy = DummyClassifier (strategy="stratified")
#Pipeline con modelo Dummy
pipeline_salida = Pipeline(steps=[('ct',transformer),('modelo_D',modelo_Dummy)])
#Preparación de salida con el pipeline de transformación considerando modelo Dummy
salida_preparada = TransformedTargetRegressor ( regressor = pipeline_salida)

In [42]:

#Se obtienen los conjuntos Dummy
XtrainDummy, XtestDummy, YtrainDummy, YtestDummy = train_test_split(Xtrain, Ytrain, train_size=0.80)

In [43]:
#Preparación variables de entrada con los imputadores
pipeline_In = Pipeline(steps=[('modelo',modelo_Dummy)])
#Preparación variables de salida con la transformación logarítimica.
pipeline_Out= TransformedTargetRegressor(regressor=pipeline_In)

#Predicción de datos con variables de entrenamiento y de prueba del conjunto Dummy
pipeline_Out.fit(XtrainDummy, YtrainDummy)
Yhat_testDummy = pipeline_Out.predict(XtestDummy)
Yhat_trainDummy= pipeline_Out.predict(XtrainDummy)
#Impresión de los resultados de Entrenamiento Dummy


print('\n-----------Resultados Dummy -- Datos de Entrenamiento---------')
names = ['accuracy','f1-score', 'precision', 'recall']
print("\n\t\t  %s\t\t%s   \t\t%s  \t\t%s" % (names[0],names[1],names[2],names[3]))
print("\t\t %.1f  \t\t%.1f   \t\t%.1f  \t\t%.1f" % (
                                            accuracy_score(YtrainDummy, Yhat_trainDummy),
                                            f1_score(YtrainDummy, Yhat_trainDummy),
                                            precision_score(YtrainDummy, Yhat_trainDummy),
                                            recall_score(YtrainDummy, Yhat_trainDummy)
                                             ),"%")
#Impresión de los resultados de Validación Dummy
print('\n-----------Resultados Dummy -- Datos de Prueba---------')
print("\n\t\t  %s\t\t%s   \t\t%s  \t\t%s" % (names[0],names[1],names[2], names[3]))
print(" \t\t %.1f  \t\t%.1f   \t\t%.1f   \t\t%.1f" % (
                                             accuracy_score(YtestDummy, Yhat_testDummy),
                                             f1_score(YtestDummy, Yhat_testDummy),
                                             precision_score(YtestDummy, Yhat_testDummy),
                                             recall_score(YtestDummy, Yhat_testDummy)
                                             ),"%")

ValueError: could not convert string to float: 'Amarillo'